# Anonimización de Datos: Airbnb Madrid

## Objetivos

Este proyecto trabaja con un dataset real que contiene los datos de los alojamientos disponibles de Airbnb para la comunidad de Madrid. El objetivo es anonimizar los datos y convertir el dataset de AirBnBMadrid a AirBnBValladolid, aplicando técnicas de anonimización y transformación de datos geográficos.

## Configuración del entorno

In [206]:
!pip install spacy
!python -m spacy download es_core_news_sm
!pip show scikit-learn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.9/12.9 MB 17.5 MB/s eta 0:00:0000:0100:01
✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_sm')
Name: scikit-learn
Version: 1.4.2
Summary: A set of python modules for machine learning and data mining
Home-page: https://scikit-learn.org
Author: 
Author-email: 
License: new BSD
Location: /opt/anaconda3/lib/python3.12/site-packages
Requires: joblib, numpy, scipy, threadpoolctl
Required-by: imbalanced-learn, librosa, mapclassify


In [207]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from faker import Faker
fake = Faker('es_ES')

# Semilla para reproducibilidad
Faker.seed(42)

## Carga de los datos
Esta vez va a ser fácil, vamos a importar los datos de un fichero csv, utilizaremos la función read_csv que nos proporciona la libreria de pandas.

In [208]:
df = pd.read_csv('dataset_airbnb_madrid.csv')

Como este dataset es muy complejo, vamos a quedarnos con un subconjunto de columnas para este análisis:

In [209]:
df = df[['id', 'listing_url', 'name', 'summary', 'price', 'weekly_price', 'zipcode', 'country', 'latitude', 'longitude']]

In [210]:
df

,id,listing_url,name,summary,price,weekly_price,zipcode,country,latitude,longitude
0,7830063,https://www.airbnb.com/rooms/7830063,Quiet room in Plaza Mayor,Room in magnificent property in the historic c...,$42.00,$300.00,28005,Spain,40.412275,-3.708718
1,9898596,https://www.airbnb.com/rooms/9898596,Homely apartment in the heart of Madrid,"Spacious apartment for up to 10 people, with a...",$135.00,NaN,28005,Spain,40.411093,-3.708985
2,15334645,https://www.airbnb.com/rooms/15334645,Piso Muy Luminoso en pleno centro de Madrid,"Lugares de interés: Casa Lucio, Cine Doré, Cal...",$81.00,NaN,28005,Spain,40.413587,-3.708945
3,1307795,https://www.airbnb.com/rooms/1307795,Rent room in the heart of Madrid,NaN,$43.00,$240.00,28013,Spain,40.419936,-3.709180
4,17410608,https://www.airbnb.com/rooms/17410608,Luxury duplex penthouse in historic building,Amazing duplex penthouse in a historic buildin...,$50.00,NaN,28005,Spain,40.410894,-3.712537
...,...,...,...,...,...,...,...,...,...,...
13330,2994471,https://www.airbnb.com/rooms/2994471,B&b Airport & IFEMA,Welcome home! :) If you are looking for a: PRI...,$32.00,$190.00,28042,Spain,40.464324,-3.591336
13331,16118534,https://www.airbnb.com/rooms/16118534,Casa con jardín cerca de aeropuerto,Habitación privada para uso individual o doble...,$40.00,NaN,28042,Spain,40.463554,-3.587371
13332,16392617,https://www.airbnb.com/rooms/16392617,"Piso con Terraza 2 personas, Aceptamos Mascotas","Piso acogedor, habitación individual para los ...",$25.00,NaN,28830,Spain,40.422470,-3.526821
13333,17840364,https://www.airbnb.com/rooms/17840364,Apartamento pozuelo,Apartamento tranquilo muy cerca de Madrid ca...,$45.00,NaN,28224,Spain,40.459903,-3.801603


## Comprensión del dataset

Una vez cargados los datos debemos inspeccionarlos, y entender que datos contiene cada una de las columnas:

Describe por cada columna:

* ¿Qué contiene?
* ¿El tipo del dato es el correcto?
* ¿Cual es el rango de los datos?
* ¿Contiene datos sensibles?
* ¿Depende de otras?

In [211]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13335 entries, 0 to 13334
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   id            13335 non-null  int64  
 1   listing_url   13335 non-null  object 
 2   name          13335 non-null  object 
 3   summary       12846 non-null  object 
 4   price         13335 non-null  object 
 5   weekly_price  3512 non-null   object 
 6   zipcode       12896 non-null  object 
 7   country       13334 non-null  object 
 8   latitude      13335 non-null  float64
 9   longitude     13335 non-null  float64
dtypes: float64(2), int64(1), object(7)
memory usage: 1.0+ MB


### id

In [234]:
print("Contiene: Identificador único de cada alojamiento")
print(f"Tipo de dato actual: {df['id'].dtype}")
print("Tipo de dato correcto: Sí, está correcto como int64")
print(f"Rango: Valores entre {df['id'].min()} y {df['id'].max()}")
print("Datos sensibles: No")
print("Dependencias: El listing_url se construye con este ID")

Contiene: Identificador único de cada alojamiento
Tipo de dato actual: int64
Tipo de dato correcto: Sí, está correcto como int64
Rango: Valores entre 18628 y 18109842
Datos sensibles: No
Dependencias: El listing_url se construye con este ID


### listing_url

In [235]:
print("Contiene: URL completa del anuncio en Airbnb")
print(f"Tipo de dato actual: {df['listing_url'].dtype}")
print("Tipo de dato correcto: Pueden quedarse como object o pasarlo a un string")
print("Rango: URLs con formato fijo 'https://www.airbnb.com/rooms/{id}'")
print("Datos sensibles: Aunque la URL en sí no es sensible, puede llevar a información sensible si se accede a ella.")
print("Dependencias: Depende del ID")

Contiene: URL completa del anuncio en Airbnb
Tipo de dato actual: object
Tipo de dato correcto: Pueden quedarse como object o pasarlo a un string
Rango: URLs con formato fijo 'https://www.airbnb.com/rooms/{id}'
Datos sensibles: Aunque la URL en sí no es sensible, puede llevar a información sensible si se accede a ella.
Dependencias: Depende del ID


### name

In [236]:
print("Contiene: Título del anuncio del alojamiento")
print(f"Tipo de dato actual: {df['name'].dtype}")
print("Tipo de dato correcto: Pueden quedarse como object o pasarlo a un string")
print(f"Rango: Textos con longitud entre {df['name'].str.len().min()} y {df['name'].str.len().max()} caracteres")
print("Datos sensibles: Sí, se podria considerar como sensible si es un nombre que existe y se podria llegar a el con un poco de investigación")
print("Dependencias: No")

Contiene: Título del anuncio del alojamiento
Tipo de dato actual: object
Tipo de dato correcto: Pueden quedarse como object o pasarlo a un string
Rango: Textos con longitud entre 1 y 75 caracteres
Datos sensibles: Sí, se podria considerar como sensible si es un nombre que existe y se podria llegar a el con un poco de investigación
Dependencias: No


### summary

In [215]:
print("Contiene: Descripción detallada del alojamiento")
print(f"Tipo de dato actual: {df['summary'].dtype}")
print("Tipo de dato correcto: Pueden quedarse como object o pasarlo a un string")
print(f"Rango: Textos con longitud entre {df['summary'].str.len().min()} y {df['summary'].str.len().max()} caracteres")
print("Datos sensibles: Sí, puede contener información y detalles específicos que permiten identificar el alojamiento")
print("Dependencias: No")

Contiene: Descripción detallada del alojamiento
Tipo de dato actual: object
Tipo de dato correcto: Pueden quedarse como object o pasarlo a un string
Rango: Textos con longitud entre 1.0 y 1000.0 caracteres
Datos sensibles: Sí, puede contener información y detalles específicos que permiten identificar el alojamiento
Dependencias: No


### price

In [216]:
print("Contiene: Precio por noche del alojamiento")
print(f"Tipo de dato actual: {df['price'].dtype}")
print("Tipo de dato correcto: float64 o int64")
print("Rango: Necesita limpieza de datos para eliminar símbolos como $ y convertir a numérico.")
print("Datos sensibles: No")
print("Dependencias: Relacionado con weekly_price")


Contiene: Precio por noche del alojamiento
Tipo de dato actual: object
Tipo de dato correcto: float64 o int64
Rango: Necesita limpieza de datos para eliminar símbolos como $ y convertir a numérico.
Datos sensibles: No
Dependencias: Relacionado con weekly_price


### weekly_price

In [217]:
print("Contiene: Precio por semana del alojamiento")
print(f"Tipo de dato actual: {df['weekly_price'].dtype}")
print("Tipo de dato correcto: float64 o int64")
print("Rango: Necesita limpieza de datos para determinar el rango real")
print("Datos sensibles: No")
print("Dependencias: Relacionado con price")


Contiene: Precio por semana del alojamiento
Tipo de dato actual: object
Tipo de dato correcto: float64 o int64
Rango: Necesita limpieza de datos para determinar el rango real
Datos sensibles: No
Dependencias: Relacionado con price


### zipcode

In [237]:
print("Contiene: Código postal del alojamiento")
print(f"Tipo de dato actual: {df['zipcode'].dtype}")
print("Tipo de dato correcto: Se puede dejar como object o pasarlo a un string")
print("Rango:")
print(df['zipcode'].unique())  # Para ver valores únicos al ser categórico
print("Datos sensibles: Sí, permite identificar la ubicación")
print("Dependencias: Relacionado con latitude/longitude")


Contiene: Código postal del alojamiento
Tipo de dato actual: object
Tipo de dato correcto: Se puede dejar como object o pasarlo a un string
Rango:
['28005' '28013' '28008' '28012' '280013' nan '28015' '28004' '28011'
 '28018' '28014' '28045' '2802\n28012' '28033' 'Madrid 28004' '28001'
 '28056' '2805' '28094' '28007' '28003' '28010' '2015' '2804' '27004'
 '28034' '29230' '20013' '28016' '25008' '27013' '28019' '28047' '28053'
 '28028' '28009' '28006' '-' '28002' '28020' '28027' '28046' '28036'
 '28002\n28002' '28030' '28039' '28040' '28060' '28029' '28290' '28105'
 '2815' '28035' ' ' '28730' '28050' '03430' '28049' '28024' '46370'
 '29012' '10100' '08015' '  ' '28038' '28023' '28025' '28660' '28044'
 '28054' '28026' '28041' '20126' '28058' '28048' '28031' '28017' '28850'
 '28' '28043' '28042' '28021' '28055' '28051' '28051\n28051' '28032'
 '28052' '28037' '28022' '84084' '28830' '28224']
Datos sensibles: Sí, permite identificar la ubicación
Dependencias: Relacionado con latitude/longit

### Country

In [238]:
print("Contiene: País del alojamiento")
print(f"Tipo de dato actual: {df['country'].dtype}")
print("Tipo de dato correcto: Pueden quedarse como object o pasarlo a un  string/categorical")
print("Rango:")
print(df['country'].unique()) 
print("Datos sensibles: No")
print("Dependencias: No")

Contiene: País del alojamiento
Tipo de dato actual: object
Tipo de dato correcto: Pueden quedarse como object o pasarlo a un  string/categorical
Rango:
['Spain' 'Cuba' nan]
Datos sensibles: No
Dependencias: No


### latitude y longitude

In [220]:
print("Contiene: Coordenadas geográficas del alojamiento")
print(f"Tipo de dato actual lat: {df['latitude'].dtype}")
print(f"Tipo de dato actual long: {df['longitude'].dtype}")
print("Tipo de dato correcto: Está bien como float64")
print("Rango:")
print(f"Latitud: {df['latitude'].min()} - {df['latitude'].max()}")
print(f"Longitud: {df['longitude'].min()} - {df['longitude'].max()}")
print("Datos sensibles: Sí, permite identificar la ubicación exacta")
print("Dependencias: Relacionado con zipcode")

Contiene: Coordenadas geográficas del alojamiento
Tipo de dato actual lat: float64
Tipo de dato actual long: float64
Tipo de dato correcto: Está bien como float64
Rango:
Latitud: 40.33188827449475 - 40.56273629126758
Longitud: -3.863907266766835 - -3.5268214016469366
Datos sensibles: Sí, permite identificar la ubicación exacta
Dependencias: Relacionado con zipcode


## Dependencias entre variables
A continuación os propongo hacer una matriz de dependencias para analizar que variables dependen entre sí y analizar que grupos de variables existen. Esto es importante de cara a la anonimización del dataset, puesto que dejar una de estas variables dependientes sin anonimizar, podría llevar a "deshacer" este proceso a personas malintencionadas. 

En esta matriz podéis marcar con una _X_ qué variables dependen entre sí. Si creéis que existen varios grupos de dependencia, asignar una marca diferente a cada uno de los grupos que consideréis. Por ejemplo, variables que dependen entre sí, podrían ser la dirección de la calle y el código postal.

| Depende de   | id | listing_url | name | summary | price | weekly_price | zipcode | country | lat y long |
|---           |--- |---          |---   |---      |---    |---           |---      |---      |---         | 
| **id**           |    |      x       |      |         |       |              |         |         |            | 
| **listing_url**  |    |             |      |         |       |              |         |         |            | 
| **name**         |    |             |      |         |       |              |         |         |            | 
| **summary**      |    |             |      |         |       |              |         |         |            | 
| **price**        |    |             |      |         |       |              |         |         |            | 
| **weekly_price** |    |             |      |         |       |              |         |         |            | 
| **zipcode**      |    |             |      |         |       |              |         |         |            | 
| **country**      |    |             |      |         |       |              |         |         |            | 
| **lat y long**   |    |             |      |         |       |              |         |         |            | 



In [222]:
"""
X: Dependencia directa (construcción/cálculo)
G: Dependencia geográfica
P: Dependencia de precio

| Depende de   | id  | listing_url| name | summary | price | weekly_price | zipcode | country | lat y long |
|--------------|-----|------------|------|---------|-------|--------------|---------|---------|------------|
| id           |     |            |      |         |       |              |         |         |            |
| listing_url  | X   |            |      |         |       |              |         |         |            |
| name         |     |            |      |         |       |              |         |         |            |
| summary      |     |            |      |         |       |              |         |         |            |
| price        |     |            |      |         |       |              |         |         |            |
| weekly_price | P   |            |      |         |       |              |         |         |            |
| zipcode      |     |            |      |         |       |              |         | G       | G          |
| country      |     |            |      |         |       |              |         |         | G          |
| lat y long   |     |            |      |         |       |              |         |         |            |
"""

"""
Grupo de Identificación (X):
	•	El listing_url se construye a partir del id.
	•	Es una dependencia unidireccional de id a listing_url.
	Al anonimizar id o listing_url, es importante considerar que dejar uno sin anonimizar puede permitir reconstruir el otro.

Grupo de Precios (P):
	•	weekly_price depende de price.
	•	El precio semanal generalmente se calcula multiplicando el precio por noche (price) por el número de noches y aplicando posibles descuentos.
	•	Es una dependencia unidireccional de price a weekly_price.
	Al anonimizar o modificar price, se debe actualizar weekly_price para mantener la coherencia y evitar inconsistencias que puedan revelar información.

Grupo de Ubicación (G):
	•	Las coordenadas geográficas (lat y long) determinan tanto el country como el zipcode.
	•	El zipcode se puede derivar de las coordenadas mediante geocodificación inversa.
	•	El country se determina a partir de las coordenadas geográficas.
	•	Es una dependencia geográfica jerárquica, donde lat y long son datos primarios, y zipcode y country dependen de ellos.
	Para proteger la privacidad y evitar la reidentificación, es crucial anonimizar o generalizar las coordenadas si también se incluyen zipcode y country.

Campos Independientes:
	•	name: No tiene dependencias directas con otros campos, pero puede contener información personal o identificativa en su contenido.
	•	summary: No tiene dependencias directas con otros campos, pero al igual que name, puede incluir detalles sensibles o identificativos.
	•	price: Aunque weekly_price depende de él, price es independiente y no depende de otras variables en este contexto.
	•	id: Es una variable independiente que no depende de otras, pero es utilizada para construir listing_url.
	•	lat y long: Son variables independientes que representan la ubicación geográfica exacta. No dependen de otras variables, pero otras variables como zipcode y country dependen de ellas.
	Aunque son independientes, los campos name y summary deben revisarse durante el proceso de anonimización para eliminar o generalizar información sensible o identificativa que puedan contener.
   
"""

'\nGrupo de Identificación (X):\n\t•\tEl listing_url se construye a partir del id.\n\t•\tEs una dependencia unidireccional de id a listing_url.\n\tAl anonimizar id o listing_url, es importante considerar que dejar uno sin anonimizar puede permitir reconstruir el otro.\n\nGrupo de Precios (P):\n\t•\tweekly_price depende de price.\n\t•\tEl precio semanal generalmente se calcula multiplicando el precio por noche (price) por el número de noches y aplicando posibles descuentos.\n\t•\tEs una dependencia unidireccional de price a weekly_price.\n\tAl anonimizar o modificar price, se debe actualizar weekly_price para mantener la coherencia y evitar inconsistencias que puedan revelar información.\n\nGrupo de Ubicación (G):\n\t•\tLas coordenadas geográficas (lat y long) determinan tanto el country como el zipcode.\n\t•\tEl zipcode se puede derivar de las coordenadas mediante geocodificación inversa.\n\t•\tEl country se determina a partir de las coordenadas geográficas.\n\t•\tEs una dependencia ge

## Estrategia de anonimización

A partir de aquí por cada grupo de variables dependientes determina cual es la estrategia de anonimización más adecuada y aplícala teniendo en cuenta las dependencias encontradas en el paso anterior.

In [223]:
"""
Grupo de Identificación (X):

Variables:
	•	id
	•	listing_url

Dependencias:
	•	listing_url se construye a partir de id.

Riesgos:
	•	Riesgo de Reidentificación Directa: Tanto id como listing_url pueden usarse para identificar directamente un anuncio específico en Airbnb.
	•	Acceso a Información Externa: Acceder a listing_url puede revelar información adicional sensible no contenida en el conjunto de datos.

Opciones de Anonimización Consideradas:
	1.	Eliminar las Columnas:
    	•	Ventajas: Elimina completamente el riesgo de reidentificación a través de estas variables.
	    •	Desventajas: Se pierde la capacidad de identificar registros de manera única dentro del conjunto de datos.
 
	2.	Reemplazar id con Nuevos Identificadores Anónimos:
	    •	Ventajas: Mantiene un identificador único para cada registro sin riesgo de reidentificación.
	    •	Desventajas: Si no se hace correctamente, podría aún existir la posibilidad de una correlación.

	3.	Modificar listing_url para Usar Nuevos IDs o Generalizarla:
	    •	Ventajas: Preserva la estructura del dato sin exponer información sensible.
	    •	Desventajas: Las URLs modificadas no conducirían a anuncios reales.

Opción Seleccionada y Argumentación:

He decidido reemplazar id con nuevos identificadores anónimos (opción 2) y modificar listing_url para reflejar estos nuevos IDs (opción 3). Esto equilibra la necesidad de mantener identificadores únicos para análisis internos y reduce significativamente el riesgo de reidentificación.
"""

# Reordenar aleatoriamente el dataset
df_anonimizado = df.sample(frac=1).reset_index(drop=True)

# Asignar nuevos IDs secuenciales
df_anonimizado['id'] = df_anonimizado.index + 1  # Iniciar desde 1

# Modificar 'listing_url' para usar los nuevos IDs
df_anonimizado['listing_url'] = df_anonimizado['id'].apply(lambda x: f"https://www.airbnb.com/rooms/{x}")
df_anonimizado

# Consideraciones:
#	Aseguro que los nuevos IDs no puedan mapearse a los IDs originales.
#	Verifico que no haya información residual en otras columnas que pueda usarse para reidentificar registros.
#	Documentaré el proceso para transparencia y reproducibilidad.

# Reordenar el Dataset:
#	Utilizo df.sample(frac=1) para reordenar aleatoriamente los registros del DataFrame.
#	reset_index(drop=True) restablece los índices y evita conservar los índices originales.
# Asignar Nuevos IDs:
#	Creo una nueva columna 'id' asignando números secuenciales empezando desde 1.
#	Esto sobrescribe la columna 'id' original, eliminando así los IDs originales.
# Modificar listing_url:
#	Actualizo la columna 'listing_url' para reflejar los nuevos IDs anónimos.
#	De esta manera, mantengo la estructura del dato sin exponer información sensible.


,id,listing_url,name,summary,price,weekly_price,zipcode,country,latitude,longitude
0,1,https://www.airbnb.com/rooms/1,Apartamento La Latina,Lugares de interés: Café Bar Delic y Museo Nac...,$99.00,NaN,NaN,Spain,40.412780,-3.711930
1,2,https://www.airbnb.com/rooms/2,ESTUDIO CÉNTRICO Y TRANQUILO,Pequeño estudio en el barrio de Chamberí equip...,$43.00,$280.00,28015,Spain,40.434239,-3.708832
2,3,https://www.airbnb.com/rooms/3,GaneshHome MadridCentr Hab.Moroco,"Habitación con cama doble, balcón a la calle, ...",$35.00,NaN,28004,Spain,40.421805,-3.702690
3,4,https://www.airbnb.com/rooms/4,"Nice room, fashionable location",NaN,$49.00,$260.00,28006,Spain,40.436368,-3.685218
4,5,https://www.airbnb.com/rooms/5,07 Single room w/private bathroom City centrer,Single room (with the posibility of an extra b...,$29.00,NaN,28013,Spain,40.417118,-3.703502
...,...,...,...,...,...,...,...,...,...,...
13330,13331,https://www.airbnb.com/rooms/13331,ATICHES: Amazing views of Madrid,Attic in the heart of Madrid. Amazing Views. I...,$70.00,NaN,28004,Spain,40.422014,-3.697071
13331,13332,https://www.airbnb.com/rooms/13332,"Luminosa habitación cerca aeropuerto, Madrid",Te va a encantar mi lugar debido a la tranquil...,$30.00,NaN,28033,Spain,40.475982,-3.657193
13332,13333,https://www.airbnb.com/rooms/13333,DOUBLE.TRIPLE ROOM WIFI CENTRAL AIR CONDITIONING,AIR CONDITIONING Double-triple room in cnetral...,$18.00,NaN,28007,Spain,40.405421,-3.675489
13333,13334,https://www.airbnb.com/rooms/13334,Apartment two travellers or couples,Luminous exterior apartment in the city center...,$60.00,$250.00,28012,Spain,40.409620,-3.698866


In [224]:
"""
Grupo de Precios (P):

Variables:
	•	price
	•	weekly_price

Dependencias:
	•	weekly_price depende de price.

Riesgos:
	•	Revelación de Información Comercial Sensible: Los precios pueden ser considerados información sensible para los anfitriones.
	•	Inconsistencias en la Anonimización: Si price y weekly_price no se anonimizan de manera coherente, pueden revelar patrones o discrepancias que ayuden a inferir los valores originales.

Opciones de Anonimización Consideradas:
	1.	Eliminar las Columnas de Precio:
	    •	Ventajas: Elimina completamente el riesgo asociado a los precios.
	    •	Desventajas: Se pierde información valiosa para análisis económicos o de mercado.
	2.	Generalizar los Precios en Rangos:
	    •	Ventajas: Preserva la utilidad del dato al mantener categorías de precios.
	    •	Desventajas: Los rangos pueden ser amplios y reducir la precisión del análisis.
	3.	Agregar Ruido Aleatorio a los Precios:
	    •	Ventajas: Dificulta la identificación de precios exactos.
	    •	Desventajas: Puede introducir inconsistencias y distorsionar análisis que requieran precisión.

Opción Seleccionada y Argumentación:

Opté por generalizar los precios en rangos (opción 2), ya que permite mantener la utilidad del dato para análisis agregados y comparativos, al tiempo que protege los valores exactos de los precios.    
"""

# Limpiar 'price' eliminando símbolos y convirtiendo a numérico
df_anonimizado['price'] = df_anonimizado['price'].replace('[\$,]', '', regex=True).astype(float)

# Definir rangos de precios
rangos_precio = [0, 50, 100, 150, 200, 250, 300, df_anonimizado['price'].max()]
etiquetas_precio = ['$0-$50', '$51-$100', '$101-$150', '$151-$200', '$201-$250', '$251-$300', '$301+']

# Categorizar 'price'
df_anonimizado['price_range'] = pd.cut(df_anonimizado['price'], bins=rangos_precio, labels=etiquetas_precio, include_lowest=True)

# Eliminar 'price' original
df_anonimizado = df_anonimizado.drop(columns=['price'])

# Ajustar 'weekly_price' en función del 'price_range'
df_anonimizado['weekly_price_range'] = df_anonimizado['price_range'].map({
    '$0-$50': '$0-$350',
    '$51-$100': '$357-$700',
    '$101-$150': '$707-$1,050',
    '$151-$200': '$1,057-$1,400',
    '$201-$250': '$1,407-$1,750',
    '$251-$300': '$1,757-$2,100',
    '$301+': '$2,107+'
})

# Eliminar 'weekly_price' original
df_anonimizado = df_anonimizado.drop(columns=['weekly_price'])

# Consideraciones:
#	Mantengo coherencia entre price y weekly_price al ajustar ambos en función de los rangos.
#	Preservo la posibilidad de realizar análisis económicos a nivel de categorías.
#	Evito revelar precios exactos que puedan ser sensibles.

df_anonimizado

<>:32: SyntaxWarning: invalid escape sequence '\$'
<>:32: SyntaxWarning: invalid escape sequence '\$'
/var/folders/0j/c164mtv91wq70n6m8ptmtmww0000gn/T/ipykernel_24848/3950403100.py:32: SyntaxWarning: invalid escape sequence '\$'
  df_anonimizado['price'] = df_anonimizado['price'].replace('[\$,]', '', regex=True).astype(float)


,id,listing_url,name,summary,zipcode,country,latitude,longitude,price_range,weekly_price_range
0,1,https://www.airbnb.com/rooms/1,Apartamento La Latina,Lugares de interés: Café Bar Delic y Museo Nac...,NaN,Spain,40.412780,-3.711930,$51-$100,$357-$700
1,2,https://www.airbnb.com/rooms/2,ESTUDIO CÉNTRICO Y TRANQUILO,Pequeño estudio en el barrio de Chamberí equip...,28015,Spain,40.434239,-3.708832,$0-$50,$0-$350
2,3,https://www.airbnb.com/rooms/3,GaneshHome MadridCentr Hab.Moroco,"Habitación con cama doble, balcón a la calle, ...",28004,Spain,40.421805,-3.702690,$0-$50,$0-$350
3,4,https://www.airbnb.com/rooms/4,"Nice room, fashionable location",NaN,28006,Spain,40.436368,-3.685218,$0-$50,$0-$350
4,5,https://www.airbnb.com/rooms/5,07 Single room w/private bathroom City centrer,Single room (with the posibility of an extra b...,28013,Spain,40.417118,-3.703502,$0-$50,$0-$350
...,...,...,...,...,...,...,...,...,...,...
13330,13331,https://www.airbnb.com/rooms/13331,ATICHES: Amazing views of Madrid,Attic in the heart of Madrid. Amazing Views. I...,28004,Spain,40.422014,-3.697071,$51-$100,$357-$700
13331,13332,https://www.airbnb.com/rooms/13332,"Luminosa habitación cerca aeropuerto, Madrid",Te va a encantar mi lugar debido a la tranquil...,28033,Spain,40.475982,-3.657193,$0-$50,$0-$350
13332,13333,https://www.airbnb.com/rooms/13333,DOUBLE.TRIPLE ROOM WIFI CENTRAL AIR CONDITIONING,AIR CONDITIONING Double-triple room in cnetral...,28007,Spain,40.405421,-3.675489,$0-$50,$0-$350
13333,13334,https://www.airbnb.com/rooms/13334,Apartment two travellers or couples,Luminous exterior apartment in the city center...,28012,Spain,40.409620,-3.698866,$51-$100,$357-$700


In [225]:
"""
Grupo de Ubicación (G):

Variables:
	•	latitude (lat)
	•	longitude (long)
	•	zipcode
	•	country

Dependencias:
	•	zipcode y country dependen de latitude y longitude.

Riesgos:
	•	Identificación Precisa de Ubicaciones: Las coordenadas exactas pueden usarse para identificar la ubicación exacta de una propiedad.
	•	Reducción del Área de Búsqueda: zipcode puede limitar significativamente el área geográfica, facilitando la identificación.

Opciones de Anonimización Consideradas:
	1.	Eliminar las Columnas de Ubicación:
	    •	Ventajas: Elimina completamente el riesgo asociado a la ubicación.
	    •	Desventajas: Se pierde información geográfica útil para análisis espaciales.
	2.	Reducir Precisión de Coordenadas:
	    •	Ventajas: Mantiene información de ubicación aproximada sin revelar ubicaciones exactas.
	    •	Desventajas: Puede afectar la precisión en análisis que requieran datos geográficos detallados.
	3.	Generalizar o Eliminar zipcode:
	    •	Ventajas: Amplía el área geográfica representada, dificultando la identificación.
	    •	Desventajas: Puede perderse especificidad en análisis que dependan del código postal.

Opción Seleccionada y Argumentación:

He decidido reducir la precisión de las coordenadas y generalizar el zipcode. Esto permite conservar información geográfica útil para análisis agregados, mientras se protege la ubicación exacta de las propiedades.
"""

# Redondear 'latitude' y 'longitude' a 1 decimal (~11 km de precisión)
df_anonimizado['latitude'] = df_anonimizado['latitude'].round(1)
df_anonimizado['longitude'] = df_anonimizado['longitude'].round(1)

# Generalizar 'zipcode' a los primeros 2 dígitos
df_anonimizado['zipcode'] = df_anonimizado['zipcode'].astype(str).str[:2]

# Consideraciones:
#	La reducción de precisión equilibra privacidad y utilidad.
#	La generalización de zipcode disminuye el riesgo de identificación manteniendo información regional.

df_anonimizado

,id,listing_url,name,summary,zipcode,country,latitude,longitude,price_range,weekly_price_range
0,1,https://www.airbnb.com/rooms/1,Apartamento La Latina,Lugares de interés: Café Bar Delic y Museo Nac...,na,Spain,40.4,-3.7,$51-$100,$357-$700
1,2,https://www.airbnb.com/rooms/2,ESTUDIO CÉNTRICO Y TRANQUILO,Pequeño estudio en el barrio de Chamberí equip...,28,Spain,40.4,-3.7,$0-$50,$0-$350
2,3,https://www.airbnb.com/rooms/3,GaneshHome MadridCentr Hab.Moroco,"Habitación con cama doble, balcón a la calle, ...",28,Spain,40.4,-3.7,$0-$50,$0-$350
3,4,https://www.airbnb.com/rooms/4,"Nice room, fashionable location",NaN,28,Spain,40.4,-3.7,$0-$50,$0-$350
4,5,https://www.airbnb.com/rooms/5,07 Single room w/private bathroom City centrer,Single room (with the posibility of an extra b...,28,Spain,40.4,-3.7,$0-$50,$0-$350
...,...,...,...,...,...,...,...,...,...,...
13330,13331,https://www.airbnb.com/rooms/13331,ATICHES: Amazing views of Madrid,Attic in the heart of Madrid. Amazing Views. I...,28,Spain,40.4,-3.7,$51-$100,$357-$700
13331,13332,https://www.airbnb.com/rooms/13332,"Luminosa habitación cerca aeropuerto, Madrid",Te va a encantar mi lugar debido a la tranquil...,28,Spain,40.5,-3.7,$0-$50,$0-$350
13332,13333,https://www.airbnb.com/rooms/13333,DOUBLE.TRIPLE ROOM WIFI CENTRAL AIR CONDITIONING,AIR CONDITIONING Double-triple room in cnetral...,28,Spain,40.4,-3.7,$0-$50,$0-$350
13333,13334,https://www.airbnb.com/rooms/13334,Apartment two travellers or couples,Luminous exterior apartment in the city center...,28,Spain,40.4,-3.7,$51-$100,$357-$700


In [233]:
"""
Variables Independientes (name y summary):

Riesgos:
	Presencia de Información Personal: Pueden contener nombres, direcciones u otros detalles específicos que identifiquen a personas o propiedades.
	Información Sensible No Estructurada: Al ser campos de texto libre, pueden incluir datos sensibles difíciles de detectar.

Opciones de Anonimización Consideradas:
	1.	Eliminar los Campos de Texto:
	    Ventajas: Elimina completamente el riesgo asociado a estos campos.
	    Desventajas: Se pierde información que podria ser valiosa dependiendo del análisis que se quiera realizar.
     
	2.	Redactar Información Sensible Utilizando Procesamiento de Lenguaje Natural (PLN):
	    Ventajas: Permite conservar el contenido general del texto mientras se elimina información sensible.
	    Desventajas: Los modelos de PLN pueden no detectar toda la información sensible, especialmente en textos informales o con errores.
     
	3.	Generalizar el Contenido:
	    Ventajas: Reemplaza detalles específicos por términos generales, reduciendo el riesgo de identificación.
	    Desventajas: Puede perderse la riqueza y especificidad del contenido original.

Opción Seleccionada y Argumentación:

	Opté por redactar la información sensible utilizando técnicas de PLN, ya que permite mantener el contenido general y utilidad de los campos de texto, 
	al tiempo que protege la información personal.

Implementación:
	Utilizo spaCy para realizar el reconocimiento de entidades nombradas (NER) y faker para generar pseudónimos que reemplazarán las entidades 
	sensibles. De esta manera, mantengo la fluidez y naturalidad del texto, reemplazando información sensible por datos ficticios pero coherentes.

"""

# Diccionarios para almacenar las sustituciones
sustituciones = {
    'PER': {},
    'LOC': {},
    'ORG': {}
}

# Diccionarios para almacenar las sustituciones
sustituciones = {
    'PER': {},
    'LOC': {},
    'ORG': {},
    'MISC': {},
    'GPE': {}
}

def anonimizar_texto_pseudo(texto):
    if pd.isnull(texto):
        # Si el texto es nulo, lo devolvemos tal cual
        return texto
    doc = nlp(texto)
    for ent in doc.ents:
        label = ent.label_
        original_text = ent.text
        # Solo procesar si la etiqueta es 'PER', 'LOC', 'ORG', 'MISC', 'GPE'
        if label in ['PER', 'LOC', 'ORG', 'MISC', 'GPE']:
            # Verificar si ya tenemos una sustitución para esta entidad
            if original_text in sustituciones[label]:
                nuevo_texto = sustituciones[label][original_text]
            else:
                # Generar un pseudónimo según el tipo de entidad
                if label == 'PER':
                    nuevo_texto = fake.name()
                elif label in ['LOC', 'GPE']:
                    nuevo_texto = fake.city()
                elif label == 'ORG':
                    nuevo_texto = fake.company()
                elif label == 'MISC':
                    nuevo_texto = 'Lugar Turístico'  # O cualquier término general
                else:
                    nuevo_texto = '###'
                # Almacenar la sustitución
                sustituciones[label][original_text] = nuevo_texto
            # Reemplazar en el texto
            texto = texto.replace(original_text, nuevo_texto)
    return texto

# Aplicar la función de anonimización a las columnas 'name' y 'summary'
df_anonimizado['name'] = df_anonimizado['name'].apply(anonimizar_texto_pseudo)
df_anonimizado['summary'] = df_anonimizado['summary'].apply(anonimizar_texto_pseudo)

df_anonimizado

,id,listing_url,name,summary,zipcode,country,latitude,longitude,price_range,weekly_price_range
0,1,https://www.airbnb.com/rooms/1,Apartamento Lugar Turístico,Lugares de interés: Lugar Turístico y Cuenca. ...,na,Spain,40.4,-3.7,$51-$100,$357-$700
1,2,https://www.airbnb.com/rooms/2,Seve Gregorio Peñas Figueras. León Y Lugar Tur...,Pequeño estudio en el barrio de Baleares equip...,28,Spain,40.4,-3.7,$0-$50,$0-$350
2,3,https://www.airbnb.com/rooms/3,Lugar Turístico.Málaga,"Ourense con cama doble, balcón a la calle, cal...",28,Spain,40.4,-3.7,$0-$50,$0-$350
3,4,https://www.airbnb.com/rooms/4,"Lugar Turístico room, fashionable location",NaN,28,Spain,40.4,-3.7,$0-$50,$0-$350
4,5,https://www.airbnb.com/rooms/5,07 Lugar Turístico room w/private Guipúzcoa ce...,Lugar Turístico (with the posibility of an ext...,28,Spain,40.4,-3.7,$0-$50,$0-$350
...,...,...,...,...,...,...,...,...,...,...
13330,13331,https://www.airbnb.com/rooms/13331,Lugar Turístico: Lugar Turístico,Lugar Turístico. Lugar Turístico. Lugar Turíst...,28,Spain,40.4,-3.7,$51-$100,$357-$700
13331,13332,https://www.airbnb.com/rooms/13332,"Luminosa habitación cerca aeropuerto, León",Te va a encantar mi lugar debido a la tranquil...,28,Spain,40.5,-3.7,$0-$50,$0-$350
13332,13333,https://www.airbnb.com/rooms/13333,Lugar Turístico ROOM Lugar Turístico CENTRAL L...,Córdoba. Lugar Turístico S.L. room in cnetral ...,28,Spain,40.4,-3.7,$0-$50,$0-$350
13333,13334,https://www.airbnb.com/rooms/13334,Baleares two travellers or couples,Luminous exterior apartment in the city center...,28,Spain,40.4,-3.7,$51-$100,$357-$700


## Conclusiones

* ¿Qué ventajas / inconvenientes le ves a esta manera de anonimizar?
* ¿Cómo podríamos mejorar los pasos de la anonimización?
* ¿Cómo podríamos evaluar que una anonimización es buena?

In [ ]:
"""
1. Ventajas e inconvenientes de esta manera de anonimizar

Ventajas:

	- Consideración de Dependencias:
		- Coherencia de Datos: Al analizar y respetar las dependencias entre variables (identificación, geografía y precio), la anonimización mantiene la integridad y consistencia del conjunto de datos.
		- Anonimización Efectiva: Al abordar las variables dependientes de manera conjunta, se reduce el riesgo de reidentificación, ya que se evita que la anonimización de una variable revele información sobre otra.

	- Preservación de Utilidad:
		- Información Generalizada pero Útil: La generalización de precios en rangos y la reducción de precisión en ubicaciones permiten realizar análisis significativos sin exponer datos sensibles.
		- Mantenimiento de Relaciones Clave: Ajustar variables dependientes como weekly_price en función de price garantiza que las relaciones internas se conserven.

Inconvenientes:

	- Complejidad de Implementación:
		- Análisis Detallado Necesario: Requiere un entendimiento de las dependencias entre variables, lo que puede aumentar el esfuerzo y tiempo invertido.
		- Riesgo de Inconsistencias: Si no se maneja correctamente, la manipulación de variables dependientes puede introducir errores o inconsistencias en los datos.

	- Pérdida de Detalle:
		- Menor Granularidad de Datos: La generalización y reducción de precisión pueden limitar la capacidad para realizar análisis que requieran información detallada.
		- Impacto en Análisis Específicos: Estudios que dependan de ubicaciones exactas o precios precisos pueden verse afectados por la anonimización.

	- Limitaciones Técnicas:
		- Modelos NER Incompletos: Las herramientas de procesamiento de lenguaje natural pueden no detectar toda la información sensible, especialmente en textos informales.
		- Dependencia de Herramientas Externas: El uso de software adicional como spaCy y faker puede complicar la implementación y requerir conocimientos más específicos en estas librerías o re-entrenar el modelo para un contexto más específico."""

In [ ]:
"""
2. Cómo mejorar los pasos de la anonimización

	a. Creo que lo principal es entender qué se quiere proteger y por qué
	
	b. Evaluar los riesgos específicos para plantear una estrategia de anonimización efectiva acorde al contexto

	c. Definir criterios claros de aceptabilidad para la anonimización

	d. Considerar el marco legal y regulatorio
 
 
	Lecciones Aprendidas de mi implementación:
   		- Campos de texto name y summary: La detección básica de entidades no fue suficiente allí se podría mejorar
   		- Códigos postales: Si bien se anonimizó el dato es posible que la utilidad del dato se vea afectada por la generalización a 2 dígitos

	Posibles Mejoras:
  		- Reentrenar modelos con datos más contextuales
  		- Validar resultados según objetivos definidos
    
    Conclusión:
    	Definir Objetivos Claros: ¿Qué se quiere proteger y por qué?

"""

In [ ]:
"""
3. Cómo evaluar que una anonimización es buena

	a. Verificar que los datos siguen siendo relevantes y útiles para el objetivo del análisis

    b. Comprobar que el proceso de anonimización es efectivo y no permite reidentificación

    c. Asegurar que la información no se comprometió o que esté en un umbral aceptable

    d. Asegurar que se cumple con el acuerdo de anonimización, leyes y normativas aplicables
	
"""